In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [7]:

data = pd.read_csv("A_Z Handwritten Data.csv").astype('float32')


In [4]:
y = data['0']
X = data.drop('0', axis=1)
X = np.array(X)
y = np.array(y)
X = X.reshape(-1, 28, 28, 1)
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train samples:", X_train.shape)
print("Test samples:", X_test.shape)

Train samples: (297960, 28, 28, 1)
Test samples: (74490, 28, 28, 1)


In [5]:
model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(26, activation='softmax')
])

c:\Users\shyam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),

    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6
    )
]
EPOCHS = 15
history = model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    batch_size=128,
    callbacks=callbacks
)
loss, acc = model.evaluate(X_test, y_test)
print(f"Accuracy: {acc * 100:.2f}%")

model.save("kaggle_az_model.h5")
print("\nModel saved as kaggle_az_model.h5")


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 424,986 (1.62 MB)

 Trainable params: 424,538 (1.62 MB)

 Non-trainable params: 448 (1.75 KB)

Epoch 1/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 177s 74ms/step - accuracy: 0.8624 - loss: 0.4993 - val_accuracy: 0.9827 - val_loss: 0.0648 - learning_rate: 0.0010
Epoch 2/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 177s 76ms/step - accuracy: 0.9764 - loss: 0.0916 - val_accuracy: 0.9861 - val_loss: 0.0535 - learning_rate: 0.0010
Epoch 3/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 164s 71ms/step - accuracy: 0.9823 - loss: 0.0677 - val_accuracy: 0.9861 - val_loss: 0.0502 - learning_rate: 0.0010
Epoch 4/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 152s 65ms/step - accuracy: 0.9854 - loss: 0.0573 - val_accuracy: 0.9903 - val_loss: 0.0388 - learning_rate: 0.0010
Epoch 5/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 161s 69ms/step - accuracy: 0.9876 - loss: 0.0486 - val_accuracy: 0.9898 - val_loss: 0.0420 - learning_rate: 0.0010
Epoch 6/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 155s 66ms/step - accuracy: 0.9881 - loss: 0.0455 - val_accuracy: 0.9886 - val_loss: 0.0465 - learning_rate: 0.0010
Epoch 7/15
2328/2328 ━━━━━━━━━━━━━━━━━━━━ 152s 65ms/step -



Model saved as kaggle_az_model.h5
